In [1]:
import numpy as np

In [2]:
nodes=150
graphRandom = np.random.rand(nodes,nodes)

In [3]:
graphRandom[graphRandom>0.25]=1
graphRandom[graphRandom<=0.5]=0

graph=graphRandom*np.transpose(graphRandom)

np.fill_diagonal(graph,0)

In [ ]:
# print(np.sum(graph,axis=1))
# graph

In [4]:
print("Number of edges ",sum(sum(graph>0))/2)
graph2 =np.array(graph)
for i in range(nodes):
    graph2[i] = graph2[i]/(np.sum(graph2[i])) 
# graph2

Number of edges  6298.0


In [5]:
def randomWalkFunction(graph,length,startNode):
    node=startNode
    itr=0
    path=[startNode]
    while (itr<length):
        validNextMoves=np.where(graph[node]>0)
        if len(validNextMoves[0])==0:
            break
        nextMove=int(np.floor(np.random.rand()*len(validNextMoves[0])))
        path.append(validNextMoves[0][nextMove])
        node=validNextMoves[0][nextMove]
        itr+=1
    return path

In [6]:
length=5000000
randomStart = int(np.floor(np.random.rand()*nodes))
randomWalk = randomWalkFunction(graph,length,randomStart)

In [8]:
windowSize=6
U=3
V=5
print(randomWalk[:10])

[109, 18, 32, 11, 84, 72, 103, 138, 146, 114]


In [9]:
#Q1 : Each ooccurance of random walk
pointersOfNodes={}

for i in range(nodes):
    pointersOfNodes[i]=[]
u=U
def occuranceOf(randomWalk,windowSize):
    context={}
    focus={}
    for i in range(nodes):
        context[i]=[]
        focus[i]=[]
    for i in range(windowSize,len(randomWalk)-windowSize):
        pointersOfNodes[randomWalk[i]].append(i)
        if randomWalk[i]==u:
            context[randomWalk[i]].append(np.array(randomWalk[i-windowSize:i+windowSize+1]))
            pointersOfNodes[randomWalk[i]].append(i)
            focus[randomWalk[i]].append(np.array(randomWalk[i-windowSize:i+windowSize+1]))
    return context,focus

context,focus=occuranceOf(randomWalk,windowSize)

nodesOccurance={}
for i in range(nodes):
    if i != u:
        continue
    count=0
    temp=context[i]
    for cont in temp:
        count+=len(cont)-1
    temp=focus[i]
    for cont in temp:
        count+=len(cont)-1
    
    nodesOccurance[i]=float(count)/len(temp)
    

In [32]:
#Q2: Probablity of walk starting at node u will vist node v
U=10
V=5

u=U
v=V
t=windowSize
def probablityOfPath(u,v,randomWalk,t,pointersOfNodes):
    count=0
    for ptr in pointersOfNodes[u]:
        if randomWalk[ptr+t]==v:
            count+=1
    return float(count)/len(pointersOfNodes[u]),float(count),len(pointersOfNodes[u])

print(probablityOfPath(u,v,randomWalk,t,pointersOfNodes))

def theoretical(u,v,t,graph2):
    oneHot=[0]*nodes
    oneHot[u]=1
    oneHot = np.transpose(oneHot)
#     np.linalg.matrix_power(graph2,t)
    return np.dot(oneHot,np.linalg.matrix_power(graph2,t))

print(theoretical(u,v,t,graph2)[v])

(0.007650180668473352, 271.0, 35424)
0.00714510485449


In [40]:
USet=[3,6,7,8,67,57]
VSet=[1,5,4,5,6,7,8]
WindowSizeArray=[2,3,4,5,6]

# USet=[3]
# VSet=[4]
# WindowSizeArray=[4]
error=0
count=0
theError=0
for u1 in USet:
    for v1 in VSet:
        for w in WindowSizeArray:
            print(u1,v1,w)
            if u1 ==v1 or u1 >= nodes or v1 >=nodes:
                continue
            u=u1
            v=v1
            t=w
            count+=1
            print(u," -> ",v,"\t",t)
            print(probablityOfPath(u,v,randomWalk,t,pointersOfNodes))
            val1=probablityOfPath(u,v,randomWalk,t,pointersOfNodes)
            the1=theoretical(u,v,t,graph2)[v]
            print(theoretical(u,v,t,graph2)[v])
            u=v1
            v=u1
            t=w
            print(u," -> ",v,"\t",t)
            print(probablityOfPath(u,v,randomWalk,t,pointersOfNodes))
            print(theoretical(u,v,t,graph2)[v])
            the2=theoretical(u,v,t,graph2)[v]
            val2=probablityOfPath(u,v,randomWalk,t,pointersOfNodes)
            error+=abs(val1[0]-val2[0])
            theError+=abs(the1-the2)
            print("----------------------------------")

3 1 2
3  ->  1 	 2
(0.007147147147147147, 476.0, 66600)
0.00708339866176
1  ->  3 	 2
(0.007098074957881072, 257.0, 36207)
0.00646068229589
----------------------------------
3 1 3
3  ->  1 	 3
(0.0075375375375375375, 502.0, 66600)
0.00731205393112
1  ->  3 	 3
(0.005634269616372525, 204.0, 36207)
0.00666923600311
----------------------------------
3 1 4
3  ->  1 	 4
(0.007867867867867868, 524.0, 66600)
0.00721852493938
1  ->  3 	 4
(0.006048554146988151, 219.0, 36207)
0.00658392934032
----------------------------------
3 1 5
3  ->  1 	 5
(0.007447447447447447, 496.0, 66600)
0.00722605868683
1  ->  3 	 5
(0.006932361145634822, 251.0, 36207)
0.0065908007803
----------------------------------
3 1 6
3  ->  1 	 6
(0.006696696696696697, 446.0, 66600)
0.00722437615193
1  ->  3 	 6
(0.007401883613665866, 268.0, 36207)
0.00658926616055
----------------------------------
3 5 2
3  ->  5 	 2
(0.007207207207207207, 480.0, 66600)
0.00721067171709
5  ->  3 	 2
(0.006785245581213526, 243.0, 35813)
0.

In [42]:
print("Average Simulation deviation between U->V and V->U ",error/count)
print("Average Theroretical deviation between U->V and V->U ",theError/count)

Average Simulation deviation between U->V and V->U  0.0005847238172234423
Average Theroretical deviation between U->V and V->U  0.000330898233687


In [25]:
#Q4B Intuition: Probablity of walk starting at node u will vist node v in left window
U=1
V=5
t=5

u=U
v=V
t=windowSize
def probablityOfPath(u,v,randomWalk,t,pointersOfNodes):
    count=0
    for ptr in pointersOfNodes[u]:
        if randomWalk[ptr-t]==v:
            count+=1
    return float(count)/len(pointersOfNodes[u]),float(count),len(pointersOfNodes[u])

print(probablityOfPath(u,v,randomWalk,t,pointersOfNodes))

def theoretical(u,v,t,graph2):
    oneHot=[0]*nodes
    oneHot[u]=1
    oneHot = np.transpose(oneHot)
#     np.linalg.matrix_power(graph2,t)
    return np.dot(oneHot,np.linalg.matrix_power(graph2,t))

print(theoretical(v,u,t,graph2)[u])

(0.008285690612312536, 300.0, 36207)
0.00722444430792


In [ ]:
randomWalk[0:5]

In [ ]:
#Q3 Expected number of times node v is visited with t steps of Node U
u=U
v=V
t=windowSize
def ExpectedSimulation(u,v,randomWalk,windowSize,pointersOfNodes):
    count=0
    for ptr in pointersOfNodes[u]:
        if v in randomWalk[ptr:ptr+windowSize+1]:
            count+=1
    return float(count)/len(pointersOfNodes[u]),float(count),len(pointersOfNodes[u])

print(ExpectedSimulation(u,v,randomWalk,t,pointersOfNodes))

def ExpectedTheoretical(u,v,T,graph2):
    count=0
    for t in range(1,T):
        count+=theoretical(u,v,t,graph2)[v]
    return count
print(ExpectedTheoretical(u,v,t,graph2))

In [ ]:
#Question 4: Skipgram softmax
t=windowSize
u=U
v=V

def beforeTStepOccurance(u,v,randomWalk,t,pointersOfNodes):
    count=0
    for ptr in pointersOfNodes[u]:
        if v == randomWalk[ptr-t]:
            count+=1
    return float(count)/len(pointersOfNodes[u]),float(count),len(pointersOfNodes[u])

def withinWindow(u,v,randomWalk,windowSize,pointersOfNodes):
    count=0
    for ptr in pointersOfNodes[u]:
        if v in randomWalk[ptr-windowSize:ptr+windowSize+1]:
            count+=np.sum(np.array(randomWalk[ptr-windowSize:ptr+windowSize+1])==v)
          
    return float(count)/(2* windowSize*len(pointersOfNodes[u])),float(count),len(pointersOfNodes[u])

prob,countUandV,uCount=withinWindow(u,v,randomWalk,t,pointersOfNodes)


withinTSteps = ExpectedTheoretical(u,v,t,graph2)
beforeTSteps =0
for t in range(1,windowSize):
    beforeTSteps += beforeTStepOccurance(u,v,randomWalk,t,pointersOfNodes)[0]
print(prob)
print(withinTSteps,beforeTSteps)
print((withinTSteps+beforeTSteps)/(2*windowSize))
print(withinTSteps/float(windowSize))

In [ ]:
USet=[3,6,7,8,67,57]
VSet=[1,5,4,5,6,7,8]
WindowSizeArray=[2,3,4,5,6]

# USet=[3]
# VSet=[4]
# WindowSizeArray=[4]

for u1 in USet:
    for v1 in VSet:
        for w in WindowSizeArray:
            print(u1,v1,w)
            if u1 ==v1 or u1 >= nodes or v1 >=nodes:
                continue
            u=u1
            v=v1
            t=w
            print("Q2")
            print("Simulation ",probablityOfPath(u,v,randomWalk,t,pointersOfNodes)[0])
            print(theoretical(u,v,t,graph2)[v])
            
            print("===========================")
            print("Q3")
            print("Simulation ",ExpectedSimulation(u,v,randomWalk,t,pointersOfNodes)[0])
            print(ExpectedTheoretical(u,v,t,graph2))
            
            print("===========================")
            print("Q4")
            
            prob,countUandV,uCount=withinWindow(u,v,randomWalk,t,pointersOfNodes)
            
            withinTSteps = ExpectedTheoretical(u,v,t,graph2)
            beforeTSteps =0
            for t in range(1,windowSize):
                beforeTSteps += beforeTStepOccurance(u,v,randomWalk,t,pointersOfNodes)[0]
            
            print ("Simulation   ", prob)
            print ("Solution1    ", (beforeTSteps+withinTSteps)/(2*t))
            print ("P(u,v,t)/t   " ,withinTSteps/float(t) )
            
            print("\n------------------------------------\n\n\n")
